# Stage1 : Get data from omdbapi

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import json

In [2]:
df = pd.read_csv("/Users/atufashireen/projects/Bollywood Movies Dataset.xlsx - Sheet1 (1) (1).csv",warn_bad_lines=True,encoding='utf-8')

/var/folders/lq/js5qhrxn52j_syhy7sq926s80000gn/T/ipykernel_4901/2437373289.py:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("/Users/atufashireen/projects/Bollywood Movies Dataset.xlsx - Sheet1 (1) (1).csv",warn_bad_lines=True,encoding='utf-8')


In [3]:
class Preprocessing():
    def __init__(self):
        pass
    def return_film(self,f): #  Drishyam (film) -> Drishyam
        v = re.sub("(\(film\))|(\(\d+ film\))|(\(\d+ .* film\))","",f)
        v = v.strip()
        return v

    def return_year(self,f): # Drishyam (2015 film) > Drishyam , 2015
        try:
            year = re.findall("(\(film\))|(\(\d+ film\))|(\(\d+ .* film\))",f)
            year = re.findall('\d+',year[0])[0]
            year = year.strip()
            return str(year)
        except Exception as e:
            return '0'

In [4]:
prep = Preprocessing()

In [5]:
df['Title'] = df['Movie Title'].apply(lambda x: prep.return_film(x))

In [ ]:
df['Year'] = 0

In [6]:
df['Year'] = df['Movie Title'].apply(lambda x: prep.return_year(x))

In [7]:
import time
class ImdbApi():
    def __init__(self, apikey:str) ->None:
        self.apikey = apikey
        self.titles_not_found = []
        self.ret_dict =  {"Title":None,
                        "ID":None,
                            "Date of release":None,
                            "Genre":None,
                            "Cast":None,
                            "Crew-writer":None,
                            "Crew-director":None,
                            "Plot summary":None,
                            "IMDB Rating":None,
                            "IMDB Votes":None,                                
                            }
    
    def create_req(self,title:str):
        req = f"http://www.omdbapi.com/?i=tt3896198&apikey={self.apikey}&t={title}"
        res_ = requests.get(req)
        return res_
    
    def return_id(self,title_year:list,):
        print("Doing...:",title_year)
        time.sleep(5)
        title = "+".join(title_year[0].split(" "))
        year = title_year[1]
        res_  = self.create_req(title)
        res = res_.text
        res = json.loads(res)
        try:
            if res_.status_code == 200:
                if str(year) !='0':
                    if res['Year'] == year:
                        self.ret_dict["Title"]=title_year[0]
                        self.ret_dict["ID"]=res['imdbID']
                        self.ret_dict["Date of release"]=res.get("Released")
                        self.ret_dict["Genre"]=res.get("Genre")
                        self.ret_dict["Cast"]=res.get("Actors")
                        self.ret_dict["Crew-writer"]=res.get("Writer")
                        self.ret_dict["Crew-director"]=res.get("Director")
                        self.ret_dict["Plot summary"]=res.get("Plot")
                        self.ret_dict["IMDB Rating"]=res.get("imdbRating")
                        self.ret_dict["IMDB Votes"]=res.get("imdbVotes")
                        return  self.ret_dict 
                    else:
                        print('Year not valid',title_year[0])
                        self.titles_not_found.append(title_year[0])
                        return self.ret_dict
                else:
                    self.ret_dict["Title"]=title_year[0]
                    self.ret_dict["ID"]=res['imdbID']
                    self.ret_dict["Date of release"]=res.get("Released")
                    self.ret_dict["Genre"]=res.get("Genre")
                    self.ret_dict["Cast"]=res.get("Actors")
                    self.ret_dict["Crew-writer"]=res.get("Writer")
                    self.ret_dict["Crew-director"]=res.get("Director")
                    self.ret_dict["Plot summary"]=res.get("Plot")
                    self.ret_dict["IMDB Rating"]=res.get("imdbRating")
                    self.ret_dict["IMDB Votes"]=res.get("imdbVotes")
                    return  self.ret_dict                           
                            
            else:
                print(f"Result Not Found in status code : {title_year[0]}")
                self.titles_not_found.append(title_year[0])
                return self.ret_dict

        except Exception as e:
            print(e)
            print(f"Result Not Found for in Except: {title_year[0]}")
            self.titles_not_found.append(title_year[0])
            return self.ret_dict
    

In [8]:
# apikey = "889bb165"

api2 = '75fbf3be'
api = ImdbApi(api2)

In [9]:
new_dict = {'Title':[],'ID':[],'Date of release':[],'Genre':[],'Cast':[],'Crew-writers':[],'Crew-director':[],'Plot summary':[],'IMDB Rating':[],'IMDB Votes':[]}

In [11]:
for i in range(0,1000):
    # time.sleep(0.01)
    print(i)
    dictns = api.return_id(list([df.iloc[i]['Title'],df.iloc[i]['Year']]))
    new_dict['Title'].append(dictns['Title'])
    new_dict['ID'].append(dictns['ID'])
    new_dict['Date of release'].append(dictns['Date of release'])
    new_dict['Genre'].append(dictns['Genre'])
    new_dict['Cast'].append(dictns['Cast'])
    new_dict['Crew-writers'].append(dictns['Crew-writer'])
    new_dict['Crew-director'].append(dictns['Crew-director'])
    new_dict['Plot summary'].append(dictns['Plot summary'])
    new_dict['IMDB Rating'].append(dictns['IMDB Rating'])
    new_dict['IMDB Votes'].append(dictns['IMDB Votes'])

100
Doing...: ['Gabbar Is Back', '0']
101
Doing...: ['Madras Cafe', '0']
102
Doing...: ['Darr', '0']
103
Doing...: ['Agneepath', '0']
104
Doing...: ['Rustom', '0']
105
Doing...: ['Bodyguard', '0']
106
Doing...: ['Kapoor & Sons', '0']
107
Doing...: ['Khosla Ka Ghosla', '0']
108
Doing...: ['Kabir Singh', '0']
109
Doing...: ['Bang Bang!', '0']
110
Doing...: ['Guru', '0']
111
Doing...: ['Raazi', '0']
112
Doing...: ['Jo Jeeta Wohi Sikandar', '0']
113
Doing...: ['Badlapur', '0']
114
Doing...: ['Masaan', '0']
115
Doing...: ['Namastey London', '0']
116
Doing...: ['Tiger Zinda Hai', '0']
117
Doing...: ['Rock On!!', '0']
118
Doing...: ['Bhool Bhulaiyaa', '0']
119
Doing...: ['Koi... Mil Gaya', '0']
120
Doing...: ['Dil To Pagal Hai', '0']
121
Doing...: ['Neerja', '0']
122
Doing...: ['Ae Dil Hai Mushkil', '0']
123
Doing...: ['Lakshya', '0']
124
Doing...: ['Pyaar Ka Punchnama', '0']
125
Doing...: ['Hindi Medium', '0']
126
Doing...: ['Parmanu: The Story of Pokhran', '0']
127
Doing...: ['The Namesake'

In [12]:
for i in new_dict.keys():
    print(len(new_dict[i]))

929
929
929
929
929
929
929
929
929
929


In [13]:
df_ = pd.DataFrame(new_dict)
df_.to_csv("scraped_titles_1.csv",index=False)

In [14]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["contelligenz"]
mycol = mydb["movies"]

In [15]:
# f = mycol.delete_many({})
# f.deleted_count

In [ ]:
for i in range(len(df_)): # len(df_)
    print(i)
    mycol.insert_one({
        "title":df_.iloc[i]['Title'],
        'id':df_.iloc[i]['ID'], 
        'release':df_.iloc[i]['Date of release'],
        'genre': df_.iloc[i]['Genre'],
        'cast':df_.iloc[i]['Cast'],
        'crew-writers':df_.iloc[i]['Crew-writers'],
        'crew-director':df_.iloc[i]['Crew-director'],
        'plot-summary':df_.iloc[i]['Plot summary'],
        'imdb-rating':df_.iloc[i]['IMDB Rating'],
        'imdb-votes':df_.iloc[i]['IMDB Votes'],
    })